
---

# 🧾 FastAPI CRUD API with SQLite and Pydantic v2

This example demonstrates a **full CRUD API** using **FastAPI**, **SQLite**, and **Pydantic v2** models for validation and serialization.

---

## ✅ Features Covered

| Feature               | Description                                 |
| --------------------- | ------------------------------------------- |
| SQLite DB Integration | Lightweight embedded DB for CRUD operations |
| Pydantic v2 Models    | Input validation & response serialization   |
| Response Models       | Clean output using `response_model`         |
| HTTPException         | Proper 404 error handling                   |

---

## 📦 Database Setup

```python
import sqlite3

def get_db():
    conn = sqlite3.connect("employee.db")
    conn.row_factory = sqlite3.Row
    return conn

with get_db() as db:
    db.execute("""
        CREATE TABLE IF NOT EXISTS employee (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            age INTEGER NOT NULL,
            city TEXT NOT NULL
        )
    """)
```

---

## 🧾 Pydantic Models

```python
from pydantic import BaseModel, Field
from typing import List

class EmployeeCreate(BaseModel):
    name: str = Field(..., examples=["John Doe"])
    age: int = Field(..., ge=0, examples=[30])
    city: str = Field(..., examples=["New York"])

class EmployeeUpdate(BaseModel):
    name: str = Field(..., examples=["Updated Name"])
    age: int = Field(..., ge=0, examples=[35])
    city: str = Field(..., examples=["Los Angeles"])

class EmployeeOut(BaseModel):
    id: int
    name: str
    age: int
    city: str

class Message(BaseModel):
    message: str
```

---

## 🚀 API Endpoints

### 🔹 Home Page

```python
@app.get("/", response_model=Message)
def home_page():
    return {"message": "Welcome to landing page of FastAPI"}
```

---

### 🔹 Create Employee

```python
@app.post("/create", response_model=Message)
def insert_record(employee: EmployeeCreate):
    with get_db() as db:
        db.execute(
            "INSERT INTO employee (name, age, city) VALUES (?, ?, ?)",
            (employee.name, employee.age, employee.city)
        )
        db.commit()
    return {"message": "Employee record inserted successfully"}
```

---

### 🔹 View All Employees

```python
@app.get("/employee/view", response_model=List[EmployeeOut])
def view_records():
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee")
        employees = cursor.fetchall()
    return [EmployeeOut.model_validate(dict(emp)) for emp in employees]
```

---

### 🔹 View Single Employee

```python
@app.get("/employee/view/{employee_id}", response_model=EmployeeOut)
def single_view_record(employee_id: int):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        employee = cursor.fetchone()
        if employee is None:
            raise HTTPException(status_code=404, detail="Employee not found")
    return EmployeeOut.model_validate(dict(employee))
```

---

### 🔹 Update Employee

```python
@app.put("/employee/update/{employee_id}", response_model=Message)
def single_update_record(employee_id: int, employee: EmployeeUpdate):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        if cursor.fetchone() is None:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute(
            "UPDATE employee SET name = ?, age = ?, city = ? WHERE id = ?",
            (employee.name, employee.age, employee.city, employee_id)
        )
        db.commit()
    return {"message": f"Employee id {employee_id} updated successfully"}
```

---

### 🔹 Delete Employee

```python
@app.delete("/employee/delete/{employee_id}", response_model=Message)
def single_record_delete(employee_id: int):
    with get_db() as db:
        cursor = db.execute("SELECT * FROM employee WHERE id = ?", (employee_id,))
        if cursor.fetchone() is None:
            raise HTTPException(status_code=404, detail="Employee not found")
        db.execute("DELETE FROM employee WHERE id = ?", (employee_id,))
        db.commit()
    return {"message": f"Employee id {employee_id} deleted successfully"}
```

---

## 🧪 To Run the App

```bash
uvicorn app_filename:app --reload
```

> Replace `app_filename` with your actual Python file name.

---
